# MCP Synthetic Data vs Historical Exchange Data Comparison

This notebook compares statistical properties of:
1. **Historical Data**: Hyperliquid L2 book quotes from S3 (24-hour period)
2. **Synthetic Data**: Aleatoric MCP-generated market data

## Goal
Demonstrate that Aleatoric's synthetic data produces similar statistical properties to real exchange data,
validating its use for backtesting and strategy development.

In [ ]:
# Required packages
# !pip install httpx pandas numpy plotly python-dotenv scipy boto3 pyarrow

In [ ]:
from __future__ import annotations

import os
import sys
import json
import ast
import math
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime, timezone, timedelta
from io import BytesIO

import boto3
import httpx
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import kurtosis, skew, ks_2samp, mannwhitneyu
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dotenv import load_dotenv
load_dotenv()

print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1. Configuration

In [ ]:
# MCP Configuration
load_dotenv()
MCP_BASE_URL = 'https://mcp.aleatoric.systems'
API_KEY = os.getenv('ALEATORIC_API_KEY')
CACHE_KEY = os.getenv('ALEATORIC_CACHE_KEY')
if not API_KEY:
    raise RuntimeError('Set ALEATORIC_API_KEY before calling MCP')


## 2. Load Historical Hyperliquid L2 Book Data from S3

In [ ]:
def load_hyperliquid_l2book_from_s3(
    bucket: str,
    prefix: str,
    symbol: str,
    date: str,
    resample_seconds: float = 5.0
) -> pd.DataFrame:
    """
    Load Hyperliquid L2 book data from S3 and parse into structured format.
    
    Args:
        bucket: S3 bucket name
        prefix: S3 key prefix
        symbol: Trading symbol (e.g., 'BTC')
        date: Date string in YYYYMMDD format
        resample_seconds: Resample interval for time-based analysis
    
    Returns:
        DataFrame with parsed L2 book data
    """
    s3 = boto3.client('s3')
    
    # Construct S3 key
    key = f"{prefix}/{symbol}/l2Book/{symbol}_l2Book_{date}.parquet"
    print(f"Loading: s3://{bucket}/{key}")
    
    # Download parquet file
    response = s3.get_object(Bucket=bucket, Key=key)
    parquet_data = BytesIO(response['Body'].read())
    
    # Read parquet
    df_raw = pd.read_parquet(parquet_data)
    print(f"Loaded {len(df_raw)} raw records")
    
    # Parse the raw field
    parsed_records = []
    parse_errors = 0
    
    for idx, row in df_raw.iterrows():
        try:
            # The raw field is already a dict (not a string)
            raw_data = row['raw']
            
            # Handle case where it might be a string
            if isinstance(raw_data, str):
                raw_data = ast.literal_eval(raw_data.replace('\n', ' '))
            
            data = raw_data.get('data', {})
            levels = data.get('levels', [])
            
            # Handle numpy arrays - convert to list
            if hasattr(levels, 'tolist'):
                levels = levels.tolist()
            
            # levels[0] = bids, levels[1] = asks
            # Each may also be a numpy array
            if len(levels) < 2:
                continue
                
            bids = levels[0]
            asks = levels[1]
            
            # Convert numpy arrays to lists if needed
            if hasattr(bids, 'tolist'):
                bids = bids.tolist()
            if hasattr(asks, 'tolist'):
                asks = asks.tolist()
            
            if not bids or not asks:
                continue
            
            # Best bid/ask
            best_bid_px = float(bids[0]['px'])
            best_bid_sz = float(bids[0]['sz'])
            best_ask_px = float(asks[0]['px'])
            best_ask_sz = float(asks[0]['sz'])
            
            # Mid price and spread
            mid_price = (best_bid_px + best_ask_px) / 2
            spread = best_ask_px - best_bid_px
            spread_bps = (spread / mid_price) * 10000
            
            # Depth analysis (sum of top N levels)
            bid_depth_5 = sum(float(b['sz']) for b in bids[:5])
            ask_depth_5 = sum(float(a['sz']) for a in asks[:5])
            
            bid_depth_10 = sum(float(b['sz']) for b in bids[:10])
            ask_depth_10 = sum(float(a['sz']) for a in asks[:10])
            
            # Imbalance
            total_depth = bid_depth_5 + ask_depth_5
            imbalance = (bid_depth_5 - ask_depth_5) / total_depth if total_depth > 0 else 0
            
            # Number of levels
            n_bid_levels = len(bids)
            n_ask_levels = len(asks)
            
            parsed_records.append({
                'timestamp': pd.to_datetime(row['timestamp']),
                'exchange_time': data.get('time', 0),
                'mid_price': mid_price,
                'best_bid': best_bid_px,
                'best_ask': best_ask_px,
                'best_bid_size': best_bid_sz,
                'best_ask_size': best_ask_sz,
                'spread': spread,
                'spread_bps': spread_bps,
                'bid_depth_5': bid_depth_5,
                'ask_depth_5': ask_depth_5,
                'bid_depth_10': bid_depth_10,
                'ask_depth_10': ask_depth_10,
                'imbalance': imbalance,
                'n_bid_levels': n_bid_levels,
                'n_ask_levels': n_ask_levels,
            })
            
        except Exception as e:
            parse_errors += 1
            if parse_errors <= 3:
                print(f"   Parse error {parse_errors}: {e}")
            continue
    
    print(f"Successfully parsed {len(parsed_records)} records ({parse_errors} errors)")
    
    if len(parsed_records) == 0:
        raise ValueError("No records were successfully parsed!")
    
    df = pd.DataFrame(parsed_records)
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)
    
    # Calculate returns
    df['returns'] = df['mid_price'].pct_change()
    df['log_returns'] = np.log(df['mid_price'] / df['mid_price'].shift(1))
    
    # Resample to specified interval for cleaner analysis
    if resample_seconds > 0:
        df_resampled = df.resample(f'{int(resample_seconds)}s').agg({
            'mid_price': 'last',
            'best_bid': 'last',
            'best_ask': 'last',
            'best_bid_size': 'mean',
            'best_ask_size': 'mean',
            'spread': 'mean',
            'spread_bps': 'mean',
            'bid_depth_5': 'mean',
            'ask_depth_5': 'mean',
            'bid_depth_10': 'mean',
            'ask_depth_10': 'mean',
            'imbalance': 'mean',
            'n_bid_levels': 'mean',
            'n_ask_levels': 'mean',
        }).dropna()
        
        # Recalculate returns on resampled data
        df_resampled['returns'] = df_resampled['mid_price'].pct_change()
        df_resampled['log_returns'] = np.log(df_resampled['mid_price'] / df_resampled['mid_price'].shift(1))
        
        print(f"Resampled to {len(df_resampled)} records at {resample_seconds}s intervals")
        return df_resampled
    
    return df

In [ ]:
# Load historical data
print("Loading historical Hyperliquid L2 book data...")
historical_df = load_hyperliquid_l2book_from_s3(
    bucket=S3_CONFIG['bucket'],
    prefix=S3_CONFIG['prefix'],
    symbol=SYMBOL,
    date=DATA_DATE,
    resample_seconds=S3_CONFIG['resample_seconds']
)

print(f"\nHistorical Data Shape: {historical_df.shape}")
print(f"Time Range: {historical_df.index.min()} to {historical_df.index.max()}")
print(f"Duration: {historical_df.index.max() - historical_df.index.min()}")
historical_df.head()

In [ ]:
# Historical data summary statistics
print("Historical Data Summary:")
print("="*60)
historical_df.describe().round(4)

## 3. Fetch MCP Synthetic Data with Matching Parameters

## MCP Synthetic Data (fetched via export_cache)


In [ ]:
# Request/validate config for MCP synthetic dataset
mcp_config = {
    "symbol": SYMBOL,
    "seed": 42,
    "duration_seconds": int(24 * 3600),
    "tick_size": 0.01,
    "lot_size": 0.001,
}

with httpx.Client(timeout=30) as client:
    validation = client.post(
        f"{MCP_BASE_URL}/mcp/config/validate",
        headers={"X-API-Key": API_KEY},
        json={"config": mcp_config},
    )
    validation.raise_for_status()
    validation = validation.json()
    cache_key = CACHE_KEY or validation.get("cache_key") or validation.get("hash")
    if not cache_key:
        raise RuntimeError("No cache_key returned. Set ALEATORIC_CACHE_KEY or use a generated dataset.")
    print(f"Using cache_key: {cache_key}")
    export = client.get(f"{MCP_BASE_URL}/mcp/caches/export/{cache_key}", headers={"X-API-Key": API_KEY})
    export.raise_for_status()
    synthetic_df = pd.read_parquet(BytesIO(export.content))
    print(f"Synthetic dataset rows: {len(synthetic_df)}")


In [ ]:
# Extract parameters from historical data to calibrate synthetic generation
historical_returns = historical_df['log_returns'].dropna()

# Annualized volatility from historical data
dt_seconds = S3_CONFIG['resample_seconds']
periods_per_year = (365 * 24 * 3600) / dt_seconds
historical_vol_annual = historical_returns.std() * np.sqrt(periods_per_year)

# Other parameters
initial_price = historical_df['mid_price'].iloc[0]
base_spread_bps = historical_df['spread_bps'].median()
num_steps = len(historical_df)

print("Calibrated Parameters from Historical Data:")
print("="*60)
print(f"Initial Price: ${initial_price:,.2f}")
print(f"Annualized Volatility: {historical_vol_annual*100:.2f}%")
print(f"Median Spread: {base_spread_bps:.2f} bps")
print(f"Number of Steps: {num_steps}")
print(f"Time Step: {dt_seconds} seconds")

In [ ]:
# Synthetic data summary
print("Synthetic Data Summary:")
print("="*60)
synthetic_df.describe().round(4)

## 4. Statistical Properties Comparison

In [ ]:
def compute_statistics(df: pd.DataFrame, name: str) -> Dict[str, float]:
    """Compute comprehensive statistics for a market data DataFrame."""
    returns = df['log_returns'].dropna()
    spreads = df['spread_bps'].dropna()
    
    # Returns statistics
    stats_dict = {
        'name': name,
        # Returns distribution
        'returns_mean_bps': returns.mean() * 10000,
        'returns_std_bps': returns.std() * 10000,
        'returns_skewness': skew(returns),
        'returns_kurtosis': kurtosis(returns),
        'returns_min_bps': returns.min() * 10000,
        'returns_max_bps': returns.max() * 10000,
        'returns_median_bps': returns.median() * 10000,
        
        # Volatility (annualized)
        'volatility_annual': returns.std() * np.sqrt(periods_per_year) * 100,
        
        # Spread statistics
        'spread_mean_bps': spreads.mean(),
        'spread_std_bps': spreads.std(),
        'spread_median_bps': spreads.median(),
        'spread_min_bps': spreads.min(),
        'spread_max_bps': spreads.max(),
        
        # Depth statistics
        'bid_depth_5_mean': df['bid_depth_5'].mean(),
        'ask_depth_5_mean': df['ask_depth_5'].mean(),
        
        # Imbalance statistics
        'imbalance_mean': df['imbalance'].mean(),
        'imbalance_std': df['imbalance'].std(),
        
        # Price statistics
        'price_mean': df['mid_price'].mean(),
        'price_std': df['mid_price'].std(),
        'price_range_pct': (df['mid_price'].max() - df['mid_price'].min()) / df['mid_price'].mean() * 100,
    }
    
    # Autocorrelation of returns (should be ~0 for efficient markets)
    if len(returns) > 10:
        stats_dict['returns_acf_1'] = returns.autocorr(lag=1) if hasattr(returns, 'autocorr') else np.nan
        
    # Autocorrelation of squared returns (volatility clustering)
    sq_returns = returns ** 2
    if len(sq_returns) > 10:
        stats_dict['sq_returns_acf_1'] = sq_returns.autocorr(lag=1) if hasattr(sq_returns, 'autocorr') else np.nan
    
    return stats_dict

In [ ]:
# Compute statistics for both datasets
hist_stats = compute_statistics(historical_df, "Historical (Hyperliquid)")
synth_stats = compute_statistics(synthetic_df, "Synthetic (MCP)")

# Create comparison DataFrame
comparison_stats = pd.DataFrame([hist_stats, synth_stats]).set_index('name').T

# Calculate percentage difference
comparison_stats['Diff %'] = (
    (comparison_stats['Synthetic (MCP)'] - comparison_stats['Historical (Hyperliquid)']) 
    / comparison_stats['Historical (Hyperliquid)'].abs() * 100
).round(2)

print("Statistical Properties Comparison:")
print("="*80)
comparison_stats.round(4)

In [ ]:
# Statistical tests for distribution similarity
hist_returns = historical_df['log_returns'].dropna()
synth_returns = synthetic_df['log_returns'].dropna()

hist_spreads = historical_df['spread_bps'].dropna()
synth_spreads = synthetic_df['spread_bps'].dropna()

# Kolmogorov-Smirnov test (distribution similarity)
ks_returns = ks_2samp(hist_returns, synth_returns)
ks_spreads = ks_2samp(hist_spreads, synth_spreads)

# Mann-Whitney U test (median comparison)
mw_returns = mannwhitneyu(hist_returns, synth_returns, alternative='two-sided')
mw_spreads = mannwhitneyu(hist_spreads, synth_spreads, alternative='two-sided')

print("Distribution Similarity Tests:")
print("="*60)
print("\nKolmogorov-Smirnov Test (H0: same distribution):")
print(f"  Returns: KS={ks_returns.statistic:.4f}, p-value={ks_returns.pvalue:.4e}")
print(f"  Spreads: KS={ks_spreads.statistic:.4f}, p-value={ks_spreads.pvalue:.4e}")

print("\nMann-Whitney U Test (H0: same median):")
print(f"  Returns: U={mw_returns.statistic:.0f}, p-value={mw_returns.pvalue:.4e}")
print(f"  Spreads: U={mw_spreads.statistic:.0f}, p-value={mw_spreads.pvalue:.4e}")

print("\nInterpretation:")
print(f"  Returns distribution match: {'Good' if ks_returns.pvalue > 0.01 else 'Differs'} (p={ks_returns.pvalue:.4f})")
print(f"  Spreads distribution match: {'Good' if ks_spreads.pvalue > 0.01 else 'Differs'} (p={ks_spreads.pvalue:.4f})")

## 5. Visualization: Side-by-Side Comparison

In [ ]:
# Price and Spread Time Series Comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Historical: Mid Price",
        "Synthetic: Mid Price",
        "Historical: Spread (bps)",
        "Synthetic: Spread (bps)"
    )
)

# Historical price
fig.add_trace(
    go.Scatter(x=historical_df.index, y=historical_df['mid_price'], 
               name="Historical Price", line=dict(color="blue", width=1)),
    row=1, col=1
)

# Synthetic price
fig.add_trace(
    go.Scatter(x=synthetic_df.index, y=synthetic_df['mid_price'], 
               name="Synthetic Price", line=dict(color="orange", width=1)),
    row=1, col=2
)

# Historical spread
fig.add_trace(
    go.Scatter(x=historical_df.index, y=historical_df['spread_bps'], 
               name="Historical Spread", line=dict(color="green", width=1)),
    row=2, col=1
)

# Synthetic spread
fig.add_trace(
    go.Scatter(x=synthetic_df.index, y=synthetic_df['spread_bps'], 
               name="Synthetic Spread", line=dict(color="purple", width=1)),
    row=2, col=2
)

fig.update_layout(height=700, title_text="Time Series Comparison: Historical vs Synthetic", showlegend=True)
fig.show()

In [ ]:
# Returns Distribution Comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Returns Distribution",
        "Q-Q Plot (Returns)",
        "Spread Distribution",
        "Depth Distribution"
    )
)

# Returns histograms
fig.add_trace(
    go.Histogram(x=hist_returns, name="Historical", opacity=0.7, 
                 histnorm="probability density", marker_color="blue"),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=synth_returns, name="Synthetic", opacity=0.7, 
                 histnorm="probability density", marker_color="orange"),
    row=1, col=1
)

# Q-Q Plot for returns
# Sort returns for Q-Q plot
hist_sorted = np.sort(hist_returns)
synth_sorted = np.sort(synth_returns)

# Resample to same length for comparison
n_points = min(len(hist_sorted), len(synth_sorted), 1000)
hist_quantiles = np.percentile(hist_returns, np.linspace(0, 100, n_points))
synth_quantiles = np.percentile(synth_returns, np.linspace(0, 100, n_points))

fig.add_trace(
    go.Scatter(x=hist_quantiles, y=synth_quantiles, mode='markers',
               name="Q-Q", marker=dict(color="green", size=3)),
    row=1, col=2
)
# Add 45-degree line
min_val = min(hist_quantiles.min(), synth_quantiles.min())
max_val = max(hist_quantiles.max(), synth_quantiles.max())
fig.add_trace(
    go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode='lines',
               name="y=x", line=dict(color="red", dash="dash")),
    row=1, col=2
)

# Spread histograms
fig.add_trace(
    go.Histogram(x=hist_spreads, name="Hist Spread", opacity=0.7,
                 histnorm="probability density", marker_color="blue"),
    row=2, col=1
)
fig.add_trace(
    go.Histogram(x=synth_spreads, name="Synth Spread", opacity=0.7,
                 histnorm="probability density", marker_color="orange"),
    row=2, col=1
)

# Depth histograms
fig.add_trace(
    go.Histogram(x=historical_df['bid_depth_5'], name="Hist Depth", opacity=0.7,
                 histnorm="probability density", marker_color="blue"),
    row=2, col=2
)
fig.add_trace(
    go.Histogram(x=synthetic_df['bid_depth_5'], name="Synth Depth", opacity=0.7,
                 histnorm="probability density", marker_color="orange"),
    row=2, col=2
)

fig.update_layout(height=700, title_text="Distribution Comparison", barmode='overlay')
fig.update_xaxes(title_text="Historical Quantiles", row=1, col=2)
fig.update_yaxes(title_text="Synthetic Quantiles", row=1, col=2)
fig.show()

In [ ]:
# Autocorrelation Comparison (Volatility Clustering)
def compute_acf(series: pd.Series, nlags: int = 30) -> np.ndarray:
    """Compute autocorrelation function."""
    acf = np.zeros(nlags)
    series = series.dropna()
    mean = series.mean()
    var = series.var()
    if var == 0:
        return acf
    for lag in range(nlags):
        if lag == 0:
            acf[lag] = 1.0
        else:
            acf[lag] = ((series.iloc[lag:].values - mean) * (series.iloc[:-lag].values - mean)).mean() / var
    return acf

# ACF of squared returns (volatility clustering indicator)
hist_sq_returns_acf = compute_acf(hist_returns ** 2, nlags=30)
synth_sq_returns_acf = compute_acf(synth_returns ** 2, nlags=30)

# ACF of spreads
hist_spread_acf = compute_acf(hist_spreads, nlags=30)
synth_spread_acf = compute_acf(synth_spreads, nlags=30)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "ACF of Squared Returns (Volatility Clustering)",
        "ACF of Spreads (Persistence)"
    )
)

lags = list(range(30))

# Squared returns ACF
fig.add_trace(
    go.Bar(x=lags, y=hist_sq_returns_acf, name="Historical", marker_color="blue", opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=lags, y=synth_sq_returns_acf, name="Synthetic", marker_color="orange", opacity=0.7),
    row=1, col=1
)

# Spread ACF
fig.add_trace(
    go.Bar(x=lags, y=hist_spread_acf, name="Hist Spread", marker_color="blue", opacity=0.7),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=lags, y=synth_spread_acf, name="Synth Spread", marker_color="orange", opacity=0.7),
    row=1, col=2
)

# Add confidence bounds
conf_bound = 1.96 / np.sqrt(len(hist_returns))
for col in [1, 2]:
    fig.add_hline(y=conf_bound, line_dash="dash", line_color="red", row=1, col=col)
    fig.add_hline(y=-conf_bound, line_dash="dash", line_color="red", row=1, col=col)

fig.update_layout(height=400, title_text="Autocorrelation Comparison", barmode='group')
fig.show()

print("\nVolatility Clustering (ACF of Squared Returns):")
print(f"  Historical ACF(1): {hist_sq_returns_acf[1]:.4f}")
print(f"  Synthetic ACF(1):  {synth_sq_returns_acf[1]:.4f}")
print(f"  Historical ACF(5): {hist_sq_returns_acf[5]:.4f}")
print(f"  Synthetic ACF(5):  {synth_sq_returns_acf[5]:.4f}")

In [ ]:
# Intraday Patterns Comparison
# Add hour column for both datasets
historical_df_hourly = historical_df.copy()
historical_df_hourly['hour'] = historical_df_hourly.index.hour

synthetic_df_hourly = synthetic_df.copy()
synthetic_df_hourly['hour'] = synthetic_df_hourly.index.hour

# Compute hourly averages
hist_hourly = historical_df_hourly.groupby('hour').agg({
    'spread_bps': 'mean',
    'bid_depth_5': 'mean',
    'log_returns': lambda x: x.std() * np.sqrt(periods_per_year) * 100  # Annualized vol
}).rename(columns={'log_returns': 'volatility'})

synth_hourly = synthetic_df_hourly.groupby('hour').agg({
    'spread_bps': 'mean',
    'bid_depth_5': 'mean',
    'log_returns': lambda x: x.std() * np.sqrt(periods_per_year) * 100
}).rename(columns={'log_returns': 'volatility'})

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Hourly Spread", "Hourly Depth", "Hourly Volatility")
)

hours = list(range(24))

# Spread by hour
fig.add_trace(
    go.Scatter(x=hours, y=hist_hourly['spread_bps'], name="Historical", 
               mode='lines+markers', line=dict(color="blue")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=hours, y=synth_hourly['spread_bps'], name="Synthetic", 
               mode='lines+markers', line=dict(color="orange")),
    row=1, col=1
)

# Depth by hour
fig.add_trace(
    go.Scatter(x=hours, y=hist_hourly['bid_depth_5'], name="Hist Depth", 
               mode='lines+markers', line=dict(color="blue")),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=hours, y=synth_hourly['bid_depth_5'], name="Synth Depth", 
               mode='lines+markers', line=dict(color="orange")),
    row=1, col=2
)

# Volatility by hour
fig.add_trace(
    go.Scatter(x=hours, y=hist_hourly['volatility'], name="Hist Vol", 
               mode='lines+markers', line=dict(color="blue")),
    row=1, col=3
)
fig.add_trace(
    go.Scatter(x=hours, y=synth_hourly['volatility'], name="Synth Vol", 
               mode='lines+markers', line=dict(color="orange")),
    row=1, col=3
)

fig.update_layout(height=400, title_text="Intraday Patterns Comparison")
fig.update_xaxes(title_text="Hour (UTC)", row=1, col=1)
fig.update_xaxes(title_text="Hour (UTC)", row=1, col=2)
fig.update_xaxes(title_text="Hour (UTC)", row=1, col=3)
fig.show()

## 6. Summary Dashboard

In [ ]:
# Create summary dashboard
fig = make_subplots(
    rows=3, cols=3,
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
        [{"colspan": 3}, None, None],
        [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]
    ],
    subplot_titles=(
        "Volatility Match", "Spread Match", "Kurtosis Match",
        "Price Overlay Comparison",
        "Returns Stats", "Spread Stats", "Depth Stats"
    ),
    row_heights=[0.2, 0.4, 0.4]
)

# Indicators
vol_match_pct = 100 - abs(comparison_stats.loc['volatility_annual', 'Diff %'])
spread_match_pct = 100 - abs(comparison_stats.loc['spread_mean_bps', 'Diff %'])
kurt_match_pct = 100 - min(abs(comparison_stats.loc['returns_kurtosis', 'Diff %']), 100)

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=vol_match_pct,
        title={'text': "Volatility"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "green" if vol_match_pct > 80 else "orange"}}
    ),
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=spread_match_pct,
        title={'text': "Spread"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "green" if spread_match_pct > 80 else "orange"}}
    ),
    row=1, col=2
)

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=kurt_match_pct,
        title={'text': "Kurtosis"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "green" if kurt_match_pct > 50 else "orange"}}
    ),
    row=1, col=3
)

# Price overlay
# Normalize prices to start at 100 for comparison
hist_norm = historical_df['mid_price'] / historical_df['mid_price'].iloc[0] * 100
synth_norm = synthetic_df['mid_price'] / synthetic_df['mid_price'].iloc[0] * 100

fig.add_trace(
    go.Scatter(x=list(range(len(hist_norm))), y=hist_norm, name="Historical", 
               line=dict(color="blue", width=1)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(len(synth_norm))), y=synth_norm, name="Synthetic", 
               line=dict(color="orange", width=1)),
    row=2, col=1
)

# Bar charts for key metrics
metrics = ['returns_std_bps', 'returns_skewness', 'returns_kurtosis']
hist_vals = [hist_stats[m] for m in metrics]
synth_vals = [synth_stats[m] for m in metrics]

fig.add_trace(
    go.Bar(x=['Std (bps)', 'Skewness', 'Kurtosis'], y=hist_vals, name="Historical", marker_color="blue"),
    row=3, col=1
)
fig.add_trace(
    go.Bar(x=['Std (bps)', 'Skewness', 'Kurtosis'], y=synth_vals, name="Synthetic", marker_color="orange"),
    row=3, col=1
)

spread_metrics = ['spread_mean_bps', 'spread_std_bps', 'spread_median_bps']
hist_spread_vals = [hist_stats[m] for m in spread_metrics]
synth_spread_vals = [synth_stats[m] for m in spread_metrics]

fig.add_trace(
    go.Bar(x=['Mean', 'Std', 'Median'], y=hist_spread_vals, name="Hist Spread", marker_color="blue"),
    row=3, col=2
)
fig.add_trace(
    go.Bar(x=['Mean', 'Std', 'Median'], y=synth_spread_vals, name="Synth Spread", marker_color="orange"),
    row=3, col=2
)

depth_metrics = ['bid_depth_5_mean', 'ask_depth_5_mean']
hist_depth_vals = [hist_stats[m] for m in depth_metrics]
synth_depth_vals = [synth_stats[m] for m in depth_metrics]

fig.add_trace(
    go.Bar(x=['Bid Depth', 'Ask Depth'], y=hist_depth_vals, name="Hist Depth", marker_color="blue"),
    row=3, col=3
)
fig.add_trace(
    go.Bar(x=['Bid Depth', 'Ask Depth'], y=synth_depth_vals, name="Synth Depth", marker_color="orange"),
    row=3, col=3
)

fig.update_layout(height=900, title_text="MCP vs Historical Data: Summary Dashboard", showlegend=True)
fig.show()

## 7. Conclusions

In [ ]:
# Final summary
print("="*80)
print("MCP SYNTHETIC DATA vs HYPERLIQUID HISTORICAL DATA COMPARISON")
print("="*80)

print(f"\nData Source: Hyperliquid L2 Book - {SYMBOL}")
print(f"Date: {DATA_DATE}")
print(f"Records: {len(historical_df)} (resampled at {S3_CONFIG['resample_seconds']}s)")

print("\n" + "-"*40)
print("KEY FINDINGS:")
print("-"*40)

print(f"\n1. VOLATILITY:")
print(f"   Historical: {hist_stats['volatility_annual']:.2f}%")
print(f"   Synthetic:  {synth_stats['volatility_annual']:.2f}%")
print(f"   Match:      {vol_match_pct:.1f}%")

print(f"\n2. SPREAD DYNAMICS:")
print(f"   Historical Mean: {hist_stats['spread_mean_bps']:.2f} bps")
print(f"   Synthetic Mean:  {synth_stats['spread_mean_bps']:.2f} bps")
print(f"   Match:           {spread_match_pct:.1f}%")

print(f"\n3. DISTRIBUTION SHAPE:")
print(f"   Historical Kurtosis: {hist_stats['returns_kurtosis']:.2f}")
print(f"   Synthetic Kurtosis:  {synth_stats['returns_kurtosis']:.2f}")
print(f"   Both exhibit fat tails: {'Yes' if hist_stats['returns_kurtosis'] > 0 and synth_stats['returns_kurtosis'] > 0 else 'No'}")

print(f"\n4. VOLATILITY CLUSTERING:")
print(f"   Historical ACF(1) sq returns: {hist_sq_returns_acf[1]:.4f}")
print(f"   Synthetic ACF(1) sq returns:  {synth_sq_returns_acf[1]:.4f}")
print(f"   Both show clustering: {'Yes' if hist_sq_returns_acf[1] > 0.05 and synth_sq_returns_acf[1] > 0.05 else 'Weak'}")

print(f"\n5. STATISTICAL TESTS:")
print(f"   KS Test (Returns): p={ks_returns.pvalue:.4f} ({'Similar' if ks_returns.pvalue > 0.01 else 'Different'})")
print(f"   KS Test (Spreads): p={ks_spreads.pvalue:.4f} ({'Similar' if ks_spreads.pvalue > 0.01 else 'Different'})")

print("\n" + "="*80)
overall_match = (vol_match_pct + spread_match_pct + kurt_match_pct) / 3
print(f"OVERALL SIMILARITY SCORE: {overall_match:.1f}%")
print("="*80)

if overall_match > 70:
    print("\nCONCLUSION: MCP synthetic data exhibits statistical properties")
    print("consistent with real Hyperliquid market data, making it suitable")
    print("for backtesting and strategy development.")
else:
    print("\nCONCLUSION: Some statistical differences exist. Consider tuning")
    print("MCP parameters to better match historical patterns.")

print("\n" + "="*80)

In [ ]:
# Export comparison results
output_dir = Path("./outputs")
output_dir.mkdir(exist_ok=True)

# Save comparison statistics
comparison_stats.to_csv(output_dir / "statistical_comparison.csv")

# Save summary
summary = {
    "symbol": SYMBOL,
    "date": DATA_DATE,
    "historical_records": len(historical_df),
    "synthetic_records": len(synthetic_df),
    "historical_volatility": hist_stats['volatility_annual'],
    "synthetic_volatility": synth_stats['volatility_annual'],
    "historical_spread_bps": hist_stats['spread_mean_bps'],
    "synthetic_spread_bps": synth_stats['spread_mean_bps'],
    "ks_test_returns_pvalue": ks_returns.pvalue,
    "ks_test_spreads_pvalue": ks_spreads.pvalue,
    "overall_similarity_pct": overall_match,
}

with open(output_dir / "comparison_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"Results exported to {output_dir.absolute()}")